<a href="https://colab.research.google.com/github/OskarKrafft/Machine-Learning-Project/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyreadstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 15.1 MB/s 


In [ ]:
import pandas as pd
import pyreadstat

In [ ]:
! git clone https://github.com/OskarKrafft/Machine-Learning-Project.git

Cloning into 'Machine-Learning-Project'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 50 (delta 14), reused 27 (delta 3), pack-reused 0
Unpacking objects: 100% (50/50), done.


In [24]:
! git init

/content/gdrive/.git: Operation not supported


In [25]:
! git pull

fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [27]:
%cd /content/gdrive/MyDrive/Colab Notebooks/Machine-Learning-Project

/content/gdrive/MyDrive/Colab Notebooks/Machine-Learning-Project


In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data, meta = pyreadstat.read_sav('../raw/1.0-EPPES.sav')

data.describe()

PyreadstatError: ignored

In [ ]:
# Drop columns with non-answered questions -----------------------------------------

index_drop = []
names_drop = []

for i in range(len(data.columns)):
  if data.count()[i] == 32524:
    index_drop.append(True)
  else:
    index_drop.append(False)
    names_drop.append(data.columns[i])

print(index_drop)
print(names_drop)

print(names_drop)

[True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, True, True, True, True, True, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, Fals

In [ ]:
%cd gdrive/My Drive/Machine Learning Project - Voting Propensity

/content/gdrive/My Drive/Machine Learning Project - Voting Propensity


In [29]:
! git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/OskarKrafft/Machine-Learning-Project
   44be49e..a1f7807  main       -> origin/main
Updating bdc015d..a1f7807
error: Your local changes to the following files would be overwritten by merge:
	Data_Cleaning.ipynb
Please commit your changes or stash them before you merge.
Aborting
